### The purpose of this workbook is to...

...process all relevant files as needed to move into central local db.

#### Overview:

<p>1) Get tables cols into a good place for loading
<p>2) Build a class that includes the following pieces of functionality:
    -  Drop/Create Table(s)
    -  Load Table(s) 
    -  Query + Preview Table(s) for Checking Purposes
<p>3) Make use of above functionality

<p>---
    
<p>The above should be flexible enough across tables. I'll start by working with at least two tables, maybe introducing more along the way...
    -  311 Complaints - the main table that houses all complaint info
    -  NYC ZIP codes - for any joins/bringing on additional geographic information
    
<p>In light of the above, the following are legitimate questions: 
    - Why build something this complex? Why not just read_csv everything later on?
<p>There are different reasons, the main one being: dealing with a ~10GB file in memory without any pre-processing is daunting on a basic machine like mine, so it will help to do some db work first. Additionally, building out functionality to make changes to db tables quickly will save potential frustration down the road and open up new opportunities if we want to bring in new tables at any point.

In [1]:
# load packages
import sqlite3, csv, re
import pandas as pd

# connect to db
con = sqlite3.connect("techtask.db")
cur = con.cursor()

In [2]:
# deal with all cols as needed

# 311 data portal here: 
#https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9

# pull complaints cols
complaints_cols = ['Unique Key', 'Created Date', 'Closed Date', 'Agency', 'Agency Name',
                   'Complaint Type', 'Descriptor', 'Location Type', 'Incident Zip',
                   'Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
                   'Intersection Street 1', 'Intersection Street 2', 'Address Type',
                   'City', 'Landmark', 'Facility Type', 'Status', 'Due Date',
                   'Resolution Description', 'Resolution Action Updated Date',
                   'Community Board', 'BBL', 'Borough', 'X Coordinate (State Plane)',
                   'Y Coordinate (State Plane)', 'Open Data Channel Type',
                   'Park Facility Name', 'Park Borough', 'Vehicle Type',
                   'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
                   'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment',
                   'Latitude', 'Longitude', 'Location']

# make complaints cols more db friendly
complaints_cols_dict = {}

for c in complaints_cols:
    
    # sub out any special characters
    c1 = re.sub("[^0-9A-Za-z\\s+]", "", c)
    complaints_cols_dict[c] = c1
    
    # join strings with underscores
    c2 = "_".join(c1.split())
    complaints_cols_dict[c] = c2
    
    # lower strings
    c3 = c2.lower()
    complaints_cols_dict[c] = c3
    
# nyc zips - something I culled together from here: 
# https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm

nyc_zips_cols = ['pk','borough','neighborhood','zip_code']

In [3]:
# create all meta-data that the class will require
# some vars will be created, then stuffed into a dict for convenience

# table ddls - won't need these if we use pd.to_sql, but keeping for record's sake

nyc_zips_ddl = "(\
pk STR,\
borough STR,\
neighborhood STR,\
zip_code STR)"

complaints_ddl = "( \
unique_key STR, \
created_date STR,\
closed_date STR,\
agency STR,\
agency_name STR, \
complaint_type STR,\
descriptor STR,\
location_type STR,\
incident_zip STR,\
incident_address STR,\
street_name STR,\
cross_street_1 STR,\
cross_street_2 STR,\
intersection_street_1 STR,\
intersection_street_2 STR,\
address_type STR,\
city STR,\
landmark STR,\
facility_type STR,\
status STR,\
due_date STR,\
resolution_description STR, \
resolution_action_updated_date STR,\
community_board STR,\
bbl STR,\
borough STR,\
x_coordinate_state_plane STR,\
y_coordinate_state_plane STR,\
open_data_channel_type STR,\
park_facility_name STR,\
park_borough STR,\
vehicle_type STR,\
taxi_company_borough STR,\
taxi_pick_up_location STR,\
bridge_highway_name STR,\
bridge_highway_direction STR,\
road_ramp STR,\
bridge_highway_segment STR,\
latitude STR,\
longitude STR,\
location STR)" 

# load funcs 

def load_nyc_zips(file_name,table,load_cols,con):
    
    # load using pandas 
    df = pd.read_csv(file_name)
    df.columns = load_cols
    df.to_sql(name = table,
              con = con,
              if_exists='fail', # shouldn't already exist
              index=False)
    
def load_complaints(file_name,table,load_cols,con):
    
    # load this large file by inserting into by chunks.
    # this helps prevent any explosions
    
    chunksize = 100000

    for df in pd.read_csv(file_name, 
                      chunksize=chunksize, 
                      iterator=True,
                      low_memory=False):
    
        df = df.rename(columns=load_cols) 
        df.to_sql(table,con, if_exists='append',index=False)

# store in central dict
meta_dict = {
    
    "nyc_zips":{"file_name":"nyc_zip_codes.csv",
                 "ddl":nyc_zips_ddl,
                 "load_cols":nyc_zips_cols,
                 "load_func":load_nyc_zips},
    
    "complaints":{"file_name":"311_data.csv",
                  "ddl":complaints_ddl,
                  "load_cols":complaints_cols_dict,
                  "load_func":load_complaints}
}

# build a separate  db dict 
db_dict = {"cur":cur,
           "con":con}

In [4]:
# build the class to store and run everything
class dbManager:

    def __init__(self,file_name,table,ddl,load_func,load_cols,db_dict):
        self.file_name = file_name
        self.table = table
        self.ddl = ddl
        self.load_func = load_func
        self.load_cols = load_cols
        self.db_dict = db_dict
       
    def drop(self):

        # this originally was drop+create, but decided to use 
        # pd.to_sql for creating/loading. So just dropping here...
        
        print("dropping " + self.table)
        self.db_dict['cur'].execute("DROP TABLE " + self.table + ";")
        self.db_dict['con'].commit()
       
    def create_and_load(self):
       
        print("creating and loading " + self.table)
        self.load_func(file_name = self.file_name,
                       load_cols = self.load_cols,
                       table = self.table,
                       con = self.db_dict['con'])
       
    def check(self):
        
        just_table_row_count = str(pd.read_sql("SELECT COUNT(*) FROM " + self.table + ";",con).iloc[0][0])
        print("row count of " + self.table + " = " + just_table_row_count)
        print("getting first 100 rows " + self.table + " from the db for you to preview...")
        print(pd.read_sql("SELECT * FROM " + self.table + " LIMIT 100;",con))

In [5]:
# use the class to interact with db
#try:
#
#    table = "nyc_zips2"
#
#    dbManager(
#        file_name = meta_dict[table]['file_name'],
#        table = table,
#        ddl = meta_dict[table]['ddl'],
#        load_cols = meta_dict[table]['load_cols'],
#        load_func = meta_dict[table]['load_func'],
#        db_dict = db_dict
#    ).drop() # change func here
#    
#except KeyError: 
#    
#    print("KeyError: check table var first. It must match a key in meta_dict")

### All tables are now loaded...

...and if, for any reason, we need to change them in any way, what we've built above should facilitate that. 
